<a href="https://www.kaggle.com/code/nnabundomr/my-image-classification-code?scriptVersionId=118684038" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#!pip install tensorflow
#!pip install keras
#!pip install --upgrade tensorflow
#!pip install tf-nightly
#!pip uninstall tensorflow-gpu
#!pip install tensorflow-gpu==2.8
#!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
#!pip install pydot
#!pip install graphviz

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#Import Module for Image classification
import os
import PIL

import matplotlib.pyplot as plt
from keras.preprocessing import image

import keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, BatchNormalization, Flatten
from keras.utils import image_dataset_from_directory
from keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam,Adamax

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from os.path import exists
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import kaggle
################################################################################################################################################
#Convert images into data-input for the model
image_size = (180,180)
batch_size = 32
train_dir = '../input/chest-xray-pneumonia/chest_xray/train'
test_dir = '../input/chest-xray-pneumonia/chest_xray/test'
val_dir = '../input/chest-xray-pneumonia/chest_xray/val'

train_data = image_dataset_from_directory(
                                        train_dir,
                                        validation_split = 0.2,
                                        subset = "training",
                                        seed = 34,
                                        label_mode = "categorical",
                                        image_size = image_size,
                                        batch_size =batch_size
)

test_data = image_dataset_from_directory(
                                        test_dir,
                                        validation_split = 0.2,
                                        subset = "validation",
                                        seed = 34,
                                        label_mode = "categorical",
                                        image_size = image_size,
                                        batch_size =batch_size
)

val_data = image_dataset_from_directory(
                                        val_dir,
                                        validation_split = 0.2,
                                        subset = "validation",
                                        seed = 34,
                                        label_mode = "categorical",
                                        image_size = image_size,
                                        batch_size =batch_size
)
################################################################################################################################################
def build_model(old_model = None):
    #Building the model with transfered Learning from Resnet and weight of my prior model.
    model = Sequential()

    pre_trained_model = ResNet50(
        include_top = False,
        input_shape = (*image_size,3),
        pooling = "avg",
        classes = len(train_data.class_names),
        weights = "imagenet"
    )

    for layer in pre_trained_model.layers : 
        layer.trainable = False

    model.add(pre_trained_model)
    model.add(Flatten())
   
    model.add(Dense(512, activation = "relu"))
    model.add(Dropout(0.25))# To reduce the effect of Overfitting
    
    model.add(Dense(2,activation = 'sigmoid'))

    if old_model != None : #A choice to use an old model's weight or not to boost accuracy
        #model_dir = '../input/model/' + old_model
        #model.load_weights(model_dir)
        pass
    else : pass

    model.compile(optimizer=Adamax(learning_rate = 0.0001),
                    loss="binary_crossentropy",
                    metrics=['accuracy'])

    return model

In [ ]:
################################################################################################################################################
#The weights of Model = rc_model.hdf5 for better accuracy.
my_model = build_model()
my_model.summary()

checkpoint_path = "../working/checkpoints/"
checkpoint_dir = os.path.dirname(checkpoint_path)

if not exists(checkpoint_dir) : 
    os.mkdir(checkpoint_dir)
    print("Create directory: " + checkpoint_dir)

In [ ]:
################################################################################################################################################
# Create a callback that saves the model's weights
epoch = 30
cp_callback = [
    tf.keras.callbacks.ModelCheckpoint(checkpoint_dir + "/Checkpoint_{epoch}.keras"), ]

In [ ]:
# Train the model with the new callback
history = my_model.fit(train_data, 
          epochs=epoch,
          validation_data=test_data,
          callbacks=cp_callback)  # Pass callback to training 
################################################################################################################################################

In [ ]:
fig1 = plt.gcf()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(
    ymin = 0.5,
    ymax = 1
)
plt.grid()
plt.title(f'Model Accuracy for {epoch}')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
keras.utils.plot_model(my_model, show_shapes=True)

In [ ]:
img = keras.preprocessing.image.load_img(val_dir+'/NORMAL/NORMAL2-IM-1431-0001.jpeg', target_size=(180,180))
img = keras.preprocessing.image.img_to_array(img)
img = np.expand_dims(img, axis=0)

In [ ]:
img = preprocess_input(img)
classes = my_model.predict(img)

score = float(classes[0][0])
print(f"This image is {100 * (1 - score):.2f}% {train_data.class_names[0]} and {100 * score:.2f}% {train_data.class_names[1]}.")

In [ ]:
#Save model
model_path = "../working/model/"
model_dir = os.path.dirname(model_path)

if not exists(model_dir) : 
    os.mkdir(model_dir)
    print("Create directory: " + model_dir)

file_count = len(os.listdir('../working/model/'))
new_fc=file_count + 1

my_model.save(f'../working/model/rc_model{new_fc}.hdf5')
################################################################################################################################################